In [1]:
# import dependencies
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report

Load in data

In [2]:
# Read in CSV to df

file_path = Path("test.csv")
df_test = pd.read_csv(file_path)

df_test

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,0x160a,CUS_0xd40,September,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,2022.0,Good,809.98,35.030402,22 Years and 9 Months,No,49.574949,236.64268203272135,Low_spent_Small_value_payments,186.26670208571772
1,0x160b,CUS_0xd40,October,Aaron Maashoh,24,821-00-0265,Scientist,19114.12,1824.843333,3,...,4.0,Good,809.98,33.053114,22 Years and 10 Months,No,49.574949,21.465380264657146,High_spent_Medium_value_payments,361.44400385378196
2,0x160c,CUS_0xd40,November,Aaron Maashoh,24,821-00-0265,Scientist,19114.12,1824.843333,3,...,4.0,Good,809.98,33.811894,NaN,No,49.574949,148.23393788500925,Low_spent_Medium_value_payments,264.67544623342997
3,0x160d,CUS_0xd40,December,Aaron Maashoh,24_,821-00-0265,Scientist,19114.12,NaN,3,...,4.0,Good,809.98,32.430559,23 Years and 0 Months,No,49.574949,39.08251089460281,High_spent_Medium_value_payments,343.82687322383634
4,0x1616,CUS_0x21b1,September,Rick Rothackerj,28,004-07-5839,_______,34847.84,3037.986667,2,...,5.0,Good,605.03,25.926822,27 Years and 3 Months,No,18.816215,39.684018417945296,High_spent_Large_value_payments,485.2984336755923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0x25fe5,CUS_0x8600,December,Sarah McBridec,4975,031-35-0942,Architect,20002.88,1929.906667,10,...,12.0,_,3571.7,34.780553,NaN,Yes,60.964772,146.48632477751087,Low_spent_Small_value_payments,275.53956951573343
49996,0x25fee,CUS_0x942c,September,Nicks,25,078-73-5990,Mechanic,39628.99,NaN,4,...,7.0,Good,502.38,27.758522,31 Years and 11 Months,NM,35.104023,181.44299902757518,Low_spent_Small_value_payments,409.39456169535066
49997,0x25fef,CUS_0x942c,October,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,7.0,Good,502.38,36.858542,32 Years and 0 Months,No,35.104023,__10000__,Low_spent_Large_value_payments,349.7263321025098
49998,0x25ff0,CUS_0x942c,November,Nicks,25,078-73-5990,Mechanic,39628.99,NaN,4,...,7.0,Good,502.38,39.139840,32 Years and 1 Months,No,35.104023,97.59857973344877,High_spent_Small_value_payments,463.23898098947717


General Cleaning

In [3]:
# dtm # of NaN values in each column
list_of_columns = ['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation', 'Annual_Income', 'Monthly_Inhand_Salary',
                   'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan', 'Delay_from_due_date',
                   'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
                   'Credit_Utilization_Ratio', 'Credit_History_Age', 'Payment_of_Min_Amount', 'Total_EMI_per_month', 'Amount_invested_monthly',
                   'Payment_Behaviour', 'Monthly_Balance']


nan_counts = df_test[list_of_columns].isna().sum()
nan_counts

ID                             0
Customer_ID                    0
Month                          0
Name                        5015
Age                            0
SSN                            0
Occupation                     0
Annual_Income                  0
Monthly_Inhand_Salary       7498
Num_Bank_Accounts              0
Num_Credit_Card                0
Interest_Rate                  0
Num_of_Loan                    0
Type_of_Loan                5704
Delay_from_due_date            0
Num_of_Delayed_Payment      3498
Changed_Credit_Limit           0
Num_Credit_Inquiries        1035
Credit_Mix                     0
Outstanding_Debt               0
Credit_Utilization_Ratio       0
Credit_History_Age          4470
Payment_of_Min_Amount          0
Total_EMI_per_month            0
Amount_invested_monthly     2271
Payment_Behaviour              0
Monthly_Balance              562
dtype: int64

In [4]:
# drop duplicate or unnecessary info for classsification model
# Name and SSN are duplicate due to Customer_ID
columns_to_drop = ['Month', 'Name', 'SSN', 'Monthly_Inhand_Salary', 'Type_of_Loan', 'Customer_ID', 'ID']
df_test.drop(columns=columns_to_drop, inplace=True)

df_test

,Age,Occupation,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,23,Scientist,19114.12,3,4,3,4,3,7,11.27,2022.0,Good,809.98,35.030402,22 Years and 9 Months,No,49.574949,236.64268203272135,Low_spent_Small_value_payments,186.26670208571772
1,24,Scientist,19114.12,3,4,3,4,3,9,13.27,4.0,Good,809.98,33.053114,22 Years and 10 Months,No,49.574949,21.465380264657146,High_spent_Medium_value_payments,361.44400385378196
2,24,Scientist,19114.12,3,4,3,4,-1,4,12.27,4.0,Good,809.98,33.811894,NaN,No,49.574949,148.23393788500925,Low_spent_Medium_value_payments,264.67544623342997
3,24_,Scientist,19114.12,3,4,3,4,4,5,11.27,4.0,Good,809.98,32.430559,23 Years and 0 Months,No,49.574949,39.08251089460281,High_spent_Medium_value_payments,343.82687322383634
4,28,_______,34847.84,2,4,6,1,3,1,5.42,5.0,Good,605.03,25.926822,27 Years and 3 Months,No,18.816215,39.684018417945296,High_spent_Large_value_payments,485.2984336755923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,4975,Architect,20002.88,10,8,29,5,33,25,18.31,12.0,_,3571.7,34.780553,NaN,Yes,60.964772,146.48632477751087,Low_spent_Small_value_payments,275.53956951573343
49996,25,Mechanic,39628.99,4,6,7,2_,20,NaN,11.5,7.0,Good,502.38,27.758522,31 Years and 11 Months,NM,35.104023,181.44299902757518,Low_spent_Small_value_payments,409.39456169535066
49997,25,Mechanic,39628.99,4,6,7,2,23,5,13.5,7.0,Good,502.38,36.858542,32 Years and 0 Months,No,35.104023,__10000__,Low_spent_Large_value_payments,349.7263321025098
49998,25,Mechanic,39628.99,4,6,7,2_,21,6_,11.5,7.0,Good,502.38,39.139840,32 Years and 1 Months,No,35.104023,97.59857973344877,High_spent_Small_value_payments,463.23898098947717


In [5]:
# drop any row with null value NaN
df_test.dropna(inplace=True)
df_test

,Age,Occupation,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,23,Scientist,19114.12,3,4,3,4,3,7,11.27,2022.0,Good,809.98,35.030402,22 Years and 9 Months,No,49.574949,236.64268203272135,Low_spent_Small_value_payments,186.26670208571772
1,24,Scientist,19114.12,3,4,3,4,3,9,13.27,4.0,Good,809.98,33.053114,22 Years and 10 Months,No,49.574949,21.465380264657146,High_spent_Medium_value_payments,361.44400385378196
3,24_,Scientist,19114.12,3,4,3,4,4,5,11.27,4.0,Good,809.98,32.430559,23 Years and 0 Months,No,49.574949,39.08251089460281,High_spent_Medium_value_payments,343.82687322383634
4,28,_______,34847.84,2,4,6,1,3,1,5.42,5.0,Good,605.03,25.926822,27 Years and 3 Months,No,18.816215,39.684018417945296,High_spent_Large_value_payments,485.2984336755923
5,28,Teacher,34847.84,2,4,6,1,3,3,5.42,5.0,Good,605.03,30.116600,27 Years and 4 Months,No,18.816215,251.62736875017606,Low_spent_Large_value_payments,303.3550833433617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49993,29,Architect,20002.88,10,8,29,5,33,25,18.31,12.0,Bad,3571.7,37.528511,6 Years and 5 Months,Yes,60.964772,71.79442082882734,Low_spent_Small_value_payments,350.23147346441687
49994,29,_______,20002.88,10,8,29,5,33,22,18.31,12.0,Bad,3571.7,27.027812,6 Years and 6 Months,Yes,60.964772,50.84684680498023,High_spent_Small_value_payments,341.179047488264
49997,25,Mechanic,39628.99,4,6,7,2,23,5,13.5,7.0,Good,502.38,36.858542,32 Years and 0 Months,No,35.104023,__10000__,Low_spent_Large_value_payments,349.7263321025098
49998,25,Mechanic,39628.99,4,6,7,2_,21,6_,11.5,7.0,Good,502.38,39.139840,32 Years and 1 Months,No,35.104023,97.59857973344877,High_spent_Small_value_payments,463.23898098947717


In [6]:
# clean Credit_Utilization_Ratio

# cast as int & round
df_test["Credit_Utilization_Ratio"] = df_test["Credit_Utilization_Ratio"].astype(float)
df_test["Credit_Utilization_Ratio"] = df_test["Credit_Utilization_Ratio"].round(2)

df_test.head(3)

,Age,Occupation,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,23,Scientist,19114.12,3,4,3,4,3,7,11.27,2022.0,Good,809.98,35.03,22 Years and 9 Months,No,49.574949,236.64268203272135,Low_spent_Small_value_payments,186.26670208571772
1,24,Scientist,19114.12,3,4,3,4,3,9,13.27,4.0,Good,809.98,33.05,22 Years and 10 Months,No,49.574949,21.465380264657146,High_spent_Medium_value_payments,361.44400385378196
3,24_,Scientist,19114.12,3,4,3,4,4,5,11.27,4.0,Good,809.98,32.43,23 Years and 0 Months,No,49.574949,39.08251089460281,High_spent_Medium_value_payments,343.82687322383634


In [7]:
# convert credit history to numerical

# Function to convert age string to total years
def convert_age_to_years(age_str):
    years, months = map(str.strip, age_str.split('Years and'))
    total_months = int(years) * 12 + int(months.split()[0])
    total_years = total_months / 12  # Convert total months to years
    return round(total_years, 1)  # Round to 1 decimal place

# Convert age strings to total years
df_test['Credit_History_Age'] = df_test['Credit_History_Age'].apply(convert_age_to_years).astype(float)

df_test.head(3)

,Age,Occupation,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,23,Scientist,19114.12,3,4,3,4,3,7,11.27,2022.0,Good,809.98,35.03,22.8,No,49.574949,236.64268203272135,Low_spent_Small_value_payments,186.26670208571772
1,24,Scientist,19114.12,3,4,3,4,3,9,13.27,4.0,Good,809.98,33.05,22.8,No,49.574949,21.465380264657146,High_spent_Medium_value_payments,361.44400385378196
3,24_,Scientist,19114.12,3,4,3,4,4,5,11.27,4.0,Good,809.98,32.43,23.0,No,49.574949,39.08251089460281,High_spent_Medium_value_payments,343.82687322383634


In [8]:
df_test

,Age,Occupation,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,23,Scientist,19114.12,3,4,3,4,3,7,11.27,2022.0,Good,809.98,35.03,22.8,No,49.574949,236.64268203272135,Low_spent_Small_value_payments,186.26670208571772
1,24,Scientist,19114.12,3,4,3,4,3,9,13.27,4.0,Good,809.98,33.05,22.8,No,49.574949,21.465380264657146,High_spent_Medium_value_payments,361.44400385378196
3,24_,Scientist,19114.12,3,4,3,4,4,5,11.27,4.0,Good,809.98,32.43,23.0,No,49.574949,39.08251089460281,High_spent_Medium_value_payments,343.82687322383634
4,28,_______,34847.84,2,4,6,1,3,1,5.42,5.0,Good,605.03,25.93,27.2,No,18.816215,39.684018417945296,High_spent_Large_value_payments,485.2984336755923
5,28,Teacher,34847.84,2,4,6,1,3,3,5.42,5.0,Good,605.03,30.12,27.3,No,18.816215,251.62736875017606,Low_spent_Large_value_payments,303.3550833433617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49993,29,Architect,20002.88,10,8,29,5,33,25,18.31,12.0,Bad,3571.7,37.53,6.4,Yes,60.964772,71.79442082882734,Low_spent_Small_value_payments,350.23147346441687
49994,29,_______,20002.88,10,8,29,5,33,22,18.31,12.0,Bad,3571.7,27.03,6.5,Yes,60.964772,50.84684680498023,High_spent_Small_value_payments,341.179047488264
49997,25,Mechanic,39628.99,4,6,7,2,23,5,13.5,7.0,Good,502.38,36.86,32.0,No,35.104023,__10000__,Low_spent_Large_value_payments,349.7263321025098
49998,25,Mechanic,39628.99,4,6,7,2_,21,6_,11.5,7.0,Good,502.38,39.14,32.1,No,35.104023,97.59857973344877,High_spent_Small_value_payments,463.23898098947717


Use Regular Expressions to Remove Typos

In [9]:
columns_to_regex = ['Annual_Income', 'Age', 'Delay_from_due_date', 'Num_of_Loan','Amount_invested_monthly', 'Monthly_Balance', 'Num_of_Delayed_Payment', 'Outstanding_Debt']

for column in columns_to_regex:
    df_test[column] = df_test[column].astype(str).str.replace('_', '')
    df_test[column] = df_test[column].astype(float)  # Convert the column back to float

df_test['Age'] = df_test['Age'].astype(int)

df_test

,Age,Occupation,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,23,Scientist,19114.12,3,4,3,4.0,3.0,7.0,11.27,2022.0,Good,809.98,35.03,22.8,No,49.574949,236.642682,Low_spent_Small_value_payments,186.266702
1,24,Scientist,19114.12,3,4,3,4.0,3.0,9.0,13.27,4.0,Good,809.98,33.05,22.8,No,49.574949,21.465380,High_spent_Medium_value_payments,361.444004
3,24,Scientist,19114.12,3,4,3,4.0,4.0,5.0,11.27,4.0,Good,809.98,32.43,23.0,No,49.574949,39.082511,High_spent_Medium_value_payments,343.826873
4,28,_______,34847.84,2,4,6,1.0,3.0,1.0,5.42,5.0,Good,605.03,25.93,27.2,No,18.816215,39.684018,High_spent_Large_value_payments,485.298434
5,28,Teacher,34847.84,2,4,6,1.0,3.0,3.0,5.42,5.0,Good,605.03,30.12,27.3,No,18.816215,251.627369,Low_spent_Large_value_payments,303.355083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49993,29,Architect,20002.88,10,8,29,5.0,33.0,25.0,18.31,12.0,Bad,3571.70,37.53,6.4,Yes,60.964772,71.794421,Low_spent_Small_value_payments,350.231473
49994,29,_______,20002.88,10,8,29,5.0,33.0,22.0,18.31,12.0,Bad,3571.70,27.03,6.5,Yes,60.964772,50.846847,High_spent_Small_value_payments,341.179047
49997,25,Mechanic,39628.99,4,6,7,2.0,23.0,5.0,13.5,7.0,Good,502.38,36.86,32.0,No,35.104023,10000.000000,Low_spent_Large_value_payments,349.726332
49998,25,Mechanic,39628.99,4,6,7,2.0,21.0,6.0,11.5,7.0,Good,502.38,39.14,32.1,No,35.104023,97.598580,High_spent_Small_value_payments,463.238981


In [10]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39156 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       39156 non-null  int64  
 1   Occupation                39156 non-null  object 
 2   Annual_Income             39156 non-null  float64
 3   Num_Bank_Accounts         39156 non-null  int64  
 4   Num_Credit_Card           39156 non-null  int64  
 5   Interest_Rate             39156 non-null  int64  
 6   Num_of_Loan               39156 non-null  float64
 7   Delay_from_due_date       39156 non-null  float64
 8   Num_of_Delayed_Payment    39156 non-null  float64
 9   Changed_Credit_Limit      39156 non-null  object 
 10  Num_Credit_Inquiries      39156 non-null  float64
 11  Credit_Mix                39156 non-null  object 
 12  Outstanding_Debt          39156 non-null  float64
 13  Credit_Utilization_Ratio  39156 non-null  float64
 14  Credit_Hist

In [11]:
# Replace nonnumeric values with numeric values (to allow casting of variables as float in next block)
df_test['Changed_Credit_Limit'] = df_test['Changed_Credit_Limit'].replace('-', '')
df_test['Changed_Credit_Limit'] = pd.to_numeric(df_test['Changed_Credit_Limit'], errors='coerce')
df_test['Changed_Credit_Limit'] = df_test['Changed_Credit_Limit'].fillna(0) 


In [12]:
# List of columns to round and cast as floats
columns_to_round = ['Num_of_Loan', 'Delay_from_due_date', 'Amount_invested_monthly', 'Monthly_Balance', 'Changed_Credit_Limit', 'Total_EMI_per_month']

# Round the columns to 2 decimal places and cast them as floats
df_test[columns_to_round] = df_test[columns_to_round].round(2).astype(float)

df_test

,Age,Occupation,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,23,Scientist,19114.12,3,4,3,4.0,3.0,7.0,11.27,2022.0,Good,809.98,35.03,22.8,No,49.57,236.64,Low_spent_Small_value_payments,186.27
1,24,Scientist,19114.12,3,4,3,4.0,3.0,9.0,13.27,4.0,Good,809.98,33.05,22.8,No,49.57,21.47,High_spent_Medium_value_payments,361.44
3,24,Scientist,19114.12,3,4,3,4.0,4.0,5.0,11.27,4.0,Good,809.98,32.43,23.0,No,49.57,39.08,High_spent_Medium_value_payments,343.83
4,28,_______,34847.84,2,4,6,1.0,3.0,1.0,5.42,5.0,Good,605.03,25.93,27.2,No,18.82,39.68,High_spent_Large_value_payments,485.30
5,28,Teacher,34847.84,2,4,6,1.0,3.0,3.0,5.42,5.0,Good,605.03,30.12,27.3,No,18.82,251.63,Low_spent_Large_value_payments,303.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49993,29,Architect,20002.88,10,8,29,5.0,33.0,25.0,18.31,12.0,Bad,3571.70,37.53,6.4,Yes,60.96,71.79,Low_spent_Small_value_payments,350.23
49994,29,_______,20002.88,10,8,29,5.0,33.0,22.0,18.31,12.0,Bad,3571.70,27.03,6.5,Yes,60.96,50.85,High_spent_Small_value_payments,341.18
49997,25,Mechanic,39628.99,4,6,7,2.0,23.0,5.0,13.50,7.0,Good,502.38,36.86,32.0,No,35.10,10000.00,Low_spent_Large_value_payments,349.73
49998,25,Mechanic,39628.99,4,6,7,2.0,21.0,6.0,11.50,7.0,Good,502.38,39.14,32.1,No,35.10,97.60,High_spent_Small_value_payments,463.24


In [13]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39156 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       39156 non-null  int64  
 1   Occupation                39156 non-null  object 
 2   Annual_Income             39156 non-null  float64
 3   Num_Bank_Accounts         39156 non-null  int64  
 4   Num_Credit_Card           39156 non-null  int64  
 5   Interest_Rate             39156 non-null  int64  
 6   Num_of_Loan               39156 non-null  float64
 7   Delay_from_due_date       39156 non-null  float64
 8   Num_of_Delayed_Payment    39156 non-null  float64
 9   Changed_Credit_Limit      39156 non-null  float64
 10  Num_Credit_Inquiries      39156 non-null  float64
 11  Credit_Mix                39156 non-null  object 
 12  Outstanding_Debt          39156 non-null  float64
 13  Credit_Utilization_Ratio  39156 non-null  float64
 14  Credit_Hist

In [14]:
df_test['Changed_Credit_Limit']

0        11.27
1        13.27
3        11.27
4         5.42
5         5.42
         ...  
49993    18.31
49994    18.31
49997    13.50
49998    11.50
49999    11.50
Name: Changed_Credit_Limit, Length: 39156, dtype: float64

Remove Outliers

In [15]:
# get a picture of bank acct outliers

# value counts
values_counted = df_test['Num_Bank_Accounts'].value_counts()

# filter out values with counts >1
filtered_counts = values_counted[values_counted > 3]

filtered_counts


Num_Bank_Accounts
 6       5062
 8       5007
 7       4982
 5       4792
 4       4789
 3       4631
 9       2157
 10      2015
 1       1760
 2       1737
 0       1691
 11        16
-1         11
 1542       4
Name: count, dtype: int64

In [16]:
# find and remove outlier, typo rows for # of bank accts

# Define the range of normal values
lower_bound = 0
upper_bound = 15

column = 'Num_Bank_Accounts'

# testing variable
within_range_df = df_test[(df_test[column] >= lower_bound) & (df_test[column] <= upper_bound)]

# used this to play with range and see where cutoff for normal values were
outliers_df = df_test[(df_test[column] < lower_bound) | (df_test[column] > upper_bound)]

# Filter the DataFrame to keep only rows with values within the specified range
cleaned_df = df_test[(df_test[column] >= lower_bound) & (df_test[column] <= upper_bound)]

df_test = cleaned_df

In [17]:
# get a picture of credit card outliers

# value counts
values_counted = df_test['Num_Credit_Card'].value_counts()

# filter out values with counts >1
filtered_counts = values_counted[values_counted > 3]

filtered_counts


Num_Credit_Card
5       7106
7       6403
6       6368
4       5451
3       5079
8       1943
10      1830
9       1815
2        880
1        843
11        34
0         10
537        4
338        4
715        4
910        4
1203       4
1250       4
41         4
571        4
591        4
211        4
Name: count, dtype: int64

In [18]:
# find and remove outlier, typo rows by # of credit cards

# Define the range of normal values
lower_bound = 0
upper_bound = 16

column = 'Num_Credit_Card'

# testing variable
within_range_df = df_test[(df_test[column] >= lower_bound) & (df_test[column] <= upper_bound)]

# used this to play with range and see where cutoff for normal values were
outliers_df = df_test[(df_test[column] < lower_bound) | (df_test[column] > upper_bound)]

# Filter the DataFrame to keep only rows with values within the specified range
cleaned_df = df_test[(df_test[column] >= lower_bound) & (df_test[column] <= upper_bound)]

df_test = cleaned_df

In [19]:
# get a picture of interest rate

column = 'Interest_Rate'

# value counts
values_counted = df_test[column].value_counts()

# filter out values with counts >1
filtered_counts = values_counted[values_counted > 3]

filtered_counts

Interest_Rate
5     1905
8     1862
6     1786
12    1741
10    1721
9     1711
7     1700
11    1635
18    1553
15    1534
20    1472
17    1446
16    1419
19    1363
3     1057
1     1020
4      954
2      947
13     885
14     866
32     666
22     655
28     633
24     626
30     621
23     619
29     611
21     611
25     601
27     590
26     567
34     550
31     545
33     538
Name: count, dtype: int64

In [20]:
#  find and remove outlier, typo rows by interest rate

# Define the range of normal values
lower_bound = 0
upper_bound = 40

column = 'Interest_Rate'

# testing variable
within_range_df = df_test[(df_test[column] >= lower_bound) & (df_test[column] <= upper_bound)]

# used this to play with range and dtm where cutoff for normal values were
outliers_df = df_test[(df_test[column] < lower_bound) | (df_test[column] > upper_bound)]

# Filter the DataFrame to keep only rows with values within the specified range
cleaned_df = df_test[(df_test[column] >= lower_bound) & (df_test[column] <= upper_bound)]

df_test = cleaned_df

df_test

,Age,Occupation,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,23,Scientist,19114.12,3,4,3,4.0,3.0,7.0,11.27,2022.0,Good,809.98,35.03,22.8,No,49.57,236.64,Low_spent_Small_value_payments,186.27
1,24,Scientist,19114.12,3,4,3,4.0,3.0,9.0,13.27,4.0,Good,809.98,33.05,22.8,No,49.57,21.47,High_spent_Medium_value_payments,361.44
3,24,Scientist,19114.12,3,4,3,4.0,4.0,5.0,11.27,4.0,Good,809.98,32.43,23.0,No,49.57,39.08,High_spent_Medium_value_payments,343.83
4,28,_______,34847.84,2,4,6,1.0,3.0,1.0,5.42,5.0,Good,605.03,25.93,27.2,No,18.82,39.68,High_spent_Large_value_payments,485.30
5,28,Teacher,34847.84,2,4,6,1.0,3.0,3.0,5.42,5.0,Good,605.03,30.12,27.3,No,18.82,251.63,Low_spent_Large_value_payments,303.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49993,29,Architect,20002.88,10,8,29,5.0,33.0,25.0,18.31,12.0,Bad,3571.70,37.53,6.4,Yes,60.96,71.79,Low_spent_Small_value_payments,350.23
49994,29,_______,20002.88,10,8,29,5.0,33.0,22.0,18.31,12.0,Bad,3571.70,27.03,6.5,Yes,60.96,50.85,High_spent_Small_value_payments,341.18
49997,25,Mechanic,39628.99,4,6,7,2.0,23.0,5.0,13.50,7.0,Good,502.38,36.86,32.0,No,35.10,10000.00,Low_spent_Large_value_payments,349.73
49998,25,Mechanic,39628.99,4,6,7,2.0,21.0,6.0,11.50,7.0,Good,502.38,39.14,32.1,No,35.10,97.60,High_spent_Small_value_payments,463.24


In [21]:
# get a picture of age

column = 'Age'

# value counts
values_counted = df_test[column].value_counts()

# filter out values with counts >1
filtered_counts = values_counted[values_counted > 3]

filtered_counts

Age
 39     1151
 35     1114
 32     1113
 44     1101
 22     1089
 27     1081
 29     1076
 36     1074
 20     1071
 37     1069
 41     1064
 28     1046
 25     1045
 26     1040
 30     1038
 24     1034
 19      997
 42      997
 33      985
 31      984
 21      979
 38      966
 23      948
 34      940
 45      936
 40      920
 43      882
 46      877
 18      607
 17      567
 15      567
 16      559
 49      521
 51      511
 47      509
 55      506
 53      503
 50      502
 54      502
 48      461
 52      457
 56      425
-500     357
 14      139
Name: count, dtype: int64

In [22]:
#  find and remove outlier, typo rows - Age

# Define the range of normal values
lower_bound = 18
upper_bound = 90

column = 'Age'

# testing variable
within_range_df = df_test[(df_test[column] >= lower_bound) & (df_test[column] <= upper_bound)]

# used this to play with range and dtm where cutoff for normal values were
outliers_df = df_test[(df_test[column] < lower_bound) | (df_test[column] > upper_bound)]

# Filter the DataFrame to keep only rows with values within the specified range
cleaned_df = df_test[(df_test[column] >= lower_bound) & (df_test[column] <= upper_bound)]

df_test = cleaned_df

df_test

,Age,Occupation,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,23,Scientist,19114.12,3,4,3,4.0,3.0,7.0,11.27,2022.0,Good,809.98,35.03,22.8,No,49.57,236.64,Low_spent_Small_value_payments,186.27
1,24,Scientist,19114.12,3,4,3,4.0,3.0,9.0,13.27,4.0,Good,809.98,33.05,22.8,No,49.57,21.47,High_spent_Medium_value_payments,361.44
3,24,Scientist,19114.12,3,4,3,4.0,4.0,5.0,11.27,4.0,Good,809.98,32.43,23.0,No,49.57,39.08,High_spent_Medium_value_payments,343.83
4,28,_______,34847.84,2,4,6,1.0,3.0,1.0,5.42,5.0,Good,605.03,25.93,27.2,No,18.82,39.68,High_spent_Large_value_payments,485.30
5,28,Teacher,34847.84,2,4,6,1.0,3.0,3.0,5.42,5.0,Good,605.03,30.12,27.3,No,18.82,251.63,Low_spent_Large_value_payments,303.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49993,29,Architect,20002.88,10,8,29,5.0,33.0,25.0,18.31,12.0,Bad,3571.70,37.53,6.4,Yes,60.96,71.79,Low_spent_Small_value_payments,350.23
49994,29,_______,20002.88,10,8,29,5.0,33.0,22.0,18.31,12.0,Bad,3571.70,27.03,6.5,Yes,60.96,50.85,High_spent_Small_value_payments,341.18
49997,25,Mechanic,39628.99,4,6,7,2.0,23.0,5.0,13.50,7.0,Good,502.38,36.86,32.0,No,35.10,10000.00,Low_spent_Large_value_payments,349.73
49998,25,Mechanic,39628.99,4,6,7,2.0,21.0,6.0,11.50,7.0,Good,502.38,39.14,32.1,No,35.10,97.60,High_spent_Small_value_payments,463.24


In [23]:
# get a picture of Payment_Behaviour

column = 'Payment_Behaviour'

# value counts
values_counted = df_test[column].value_counts()

# filter out values with counts >1
filtered_counts = values_counted[values_counted > 3]

filtered_counts

Payment_Behaviour
Low_spent_Small_value_payments      8388
High_spent_Medium_value_payments    6120
High_spent_Large_value_payments     4808
Low_spent_Medium_value_payments     4656
High_spent_Small_value_payments     3941
Low_spent_Large_value_payments      3635
!@9#%8                              2573
Name: count, dtype: int64

In [24]:
df_test = df_test[df_test['Payment_Behaviour'] != '!@9#%8']

In [25]:
# get a picture of Amount_invested_monthly

column = 'Amount_invested_monthly'

# value counts
values_counted = df_test[column].value_counts()

# filter out values with counts >1
filtered_counts = values_counted[values_counted > 3]

filtered_counts

Amount_invested_monthly
10000.00    1456
0.00          63
98.41          7
92.56          6
86.25          6
            ... 
61.18          4
91.32          4
101.80         4
22.38          4
74.81          4
Name: count, Length: 620, dtype: int64

In [26]:
#  find and remove outlier, typo rows - Amount_invested_monthly

# Define the range of normal values
lower_bound = 0
upper_bound = 9000

column = 'Amount_invested_monthly'

# testing variable
within_range_df = df_test[(df_test[column] >= lower_bound) & (df_test[column] <= upper_bound)]

# used this to play with range and dtm where cutoff for normal values were
outliers_df = df_test[(df_test[column] < lower_bound) | (df_test[column] > upper_bound)]

# Filter the DataFrame to keep only rows with values within the specified range
cleaned_df = df_test[(df_test[column] >= lower_bound) & (df_test[column] <= upper_bound)]

df_test = cleaned_df

df_test

,Age,Occupation,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,23,Scientist,19114.12,3,4,3,4.0,3.0,7.0,11.27,2022.0,Good,809.98,35.03,22.8,No,49.57,236.64,Low_spent_Small_value_payments,186.27
1,24,Scientist,19114.12,3,4,3,4.0,3.0,9.0,13.27,4.0,Good,809.98,33.05,22.8,No,49.57,21.47,High_spent_Medium_value_payments,361.44
3,24,Scientist,19114.12,3,4,3,4.0,4.0,5.0,11.27,4.0,Good,809.98,32.43,23.0,No,49.57,39.08,High_spent_Medium_value_payments,343.83
4,28,_______,34847.84,2,4,6,1.0,3.0,1.0,5.42,5.0,Good,605.03,25.93,27.2,No,18.82,39.68,High_spent_Large_value_payments,485.30
5,28,Teacher,34847.84,2,4,6,1.0,3.0,3.0,5.42,5.0,Good,605.03,30.12,27.3,No,18.82,251.63,Low_spent_Large_value_payments,303.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49992,29,Architect,20002.88,10,8,29,5.0,33.0,25.0,18.31,9.0,Bad,3571.70,32.39,6.3,Yes,60.96,107.21,Low_spent_Small_value_payments,314.82
49993,29,Architect,20002.88,10,8,29,5.0,33.0,25.0,18.31,12.0,Bad,3571.70,37.53,6.4,Yes,60.96,71.79,Low_spent_Small_value_payments,350.23
49994,29,_______,20002.88,10,8,29,5.0,33.0,22.0,18.31,12.0,Bad,3571.70,27.03,6.5,Yes,60.96,50.85,High_spent_Small_value_payments,341.18
49998,25,Mechanic,39628.99,4,6,7,2.0,21.0,6.0,11.50,7.0,Good,502.38,39.14,32.1,No,35.10,97.60,High_spent_Small_value_payments,463.24


In [27]:
# get a picture of Num_of_Loan

column = 'Num_of_Loan'

# value counts
values_counted = df_test[column].value_counts()

# filter out values with counts >1
filtered_counts = values_counted[values_counted > 3]

filtered_counts

Num_of_Loan
 2.0      4617
 3.0      4603
 4.0      4467
 0.0      3489
 1.0      3342
 6.0      2188
 5.0      2093
 7.0      2047
-100.0    1169
 9.0      1014
 8.0       912
Name: count, dtype: int64

In [28]:
#  find and remove outlier, typo rows - Num_of_Loan

# Define the range of normal values
lower_bound = 0
upper_bound = 10

column = 'Num_of_Loan'

# testing variable
within_range_df = df_test[(df_test[column] >= lower_bound) & (df_test[column] <= upper_bound)]

# used this to play with range and dtm where cutoff for normal values were
outliers_df = df_test[(df_test[column] < lower_bound) | (df_test[column] > upper_bound)]

# Filter the DataFrame to keep only rows with values within the specified range
cleaned_df = df_test[(df_test[column] >= lower_bound) & (df_test[column] <= upper_bound)]

df_test = cleaned_df

df_test

,Age,Occupation,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,23,Scientist,19114.12,3,4,3,4.0,3.0,7.0,11.27,2022.0,Good,809.98,35.03,22.8,No,49.57,236.64,Low_spent_Small_value_payments,186.27
1,24,Scientist,19114.12,3,4,3,4.0,3.0,9.0,13.27,4.0,Good,809.98,33.05,22.8,No,49.57,21.47,High_spent_Medium_value_payments,361.44
3,24,Scientist,19114.12,3,4,3,4.0,4.0,5.0,11.27,4.0,Good,809.98,32.43,23.0,No,49.57,39.08,High_spent_Medium_value_payments,343.83
4,28,_______,34847.84,2,4,6,1.0,3.0,1.0,5.42,5.0,Good,605.03,25.93,27.2,No,18.82,39.68,High_spent_Large_value_payments,485.30
5,28,Teacher,34847.84,2,4,6,1.0,3.0,3.0,5.42,5.0,Good,605.03,30.12,27.3,No,18.82,251.63,Low_spent_Large_value_payments,303.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49992,29,Architect,20002.88,10,8,29,5.0,33.0,25.0,18.31,9.0,Bad,3571.70,32.39,6.3,Yes,60.96,107.21,Low_spent_Small_value_payments,314.82
49993,29,Architect,20002.88,10,8,29,5.0,33.0,25.0,18.31,12.0,Bad,3571.70,37.53,6.4,Yes,60.96,71.79,Low_spent_Small_value_payments,350.23
49994,29,_______,20002.88,10,8,29,5.0,33.0,22.0,18.31,12.0,Bad,3571.70,27.03,6.5,Yes,60.96,50.85,High_spent_Small_value_payments,341.18
49998,25,Mechanic,39628.99,4,6,7,2.0,21.0,6.0,11.50,7.0,Good,502.38,39.14,32.1,No,35.10,97.60,High_spent_Small_value_payments,463.24


In [29]:
# get a picture of Num_of_Delayed_Payment

column = 'Num_of_Delayed_Payment'

# value counts
values_counted = df_test[column].value_counts()

# filter out values with counts >1
filtered_counts = values_counted[values_counted > 3]

filtered_counts

Num_of_Delayed_Payment
 19.0    1706
 12.0    1624
 10.0    1618
 15.0    1606
 18.0    1606
 16.0    1587
 11.0    1575
 17.0    1567
 9.0     1555
 8.0     1549
 20.0    1514
 14.0    1268
 13.0    1240
 21.0     816
 7.0      778
 6.0      737
 22.0     689
 5.0      651
 3.0      616
 23.0     608
 4.0      601
 2.0      589
 1.0      556
 0.0      556
 24.0     497
 25.0     469
 26.0      84
-1.0       82
-2.0       66
 27.0      63
 28.0      35
-3.0       31
Name: count, dtype: int64

In [30]:
#  find and remove outlier, typo rows - Num_of_Delayed_Payment

# Define the range of normal values
lower_bound = 0
upper_bound = 30
column = 'Num_of_Delayed_Payment'

# testing variable
within_range_df = df_test[(df_test[column] >= lower_bound) & (df_test[column] <= upper_bound)]

# used this to play with range and dtm where cutoff for normal values were
outliers_df = df_test[(df_test[column] < lower_bound) | (df_test[column] > upper_bound)]

# Filter the DataFrame to keep only rows with values within the specified range
cleaned_df = df_test[(df_test[column] >= lower_bound) & (df_test[column] <= upper_bound)]

df_test = cleaned_df

df_test

,Age,Occupation,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,23,Scientist,19114.12,3,4,3,4.0,3.0,7.0,11.27,2022.0,Good,809.98,35.03,22.8,No,49.57,236.64,Low_spent_Small_value_payments,186.27
1,24,Scientist,19114.12,3,4,3,4.0,3.0,9.0,13.27,4.0,Good,809.98,33.05,22.8,No,49.57,21.47,High_spent_Medium_value_payments,361.44
3,24,Scientist,19114.12,3,4,3,4.0,4.0,5.0,11.27,4.0,Good,809.98,32.43,23.0,No,49.57,39.08,High_spent_Medium_value_payments,343.83
4,28,_______,34847.84,2,4,6,1.0,3.0,1.0,5.42,5.0,Good,605.03,25.93,27.2,No,18.82,39.68,High_spent_Large_value_payments,485.30
5,28,Teacher,34847.84,2,4,6,1.0,3.0,3.0,5.42,5.0,Good,605.03,30.12,27.3,No,18.82,251.63,Low_spent_Large_value_payments,303.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49992,29,Architect,20002.88,10,8,29,5.0,33.0,25.0,18.31,9.0,Bad,3571.70,32.39,6.3,Yes,60.96,107.21,Low_spent_Small_value_payments,314.82
49993,29,Architect,20002.88,10,8,29,5.0,33.0,25.0,18.31,12.0,Bad,3571.70,37.53,6.4,Yes,60.96,71.79,Low_spent_Small_value_payments,350.23
49994,29,_______,20002.88,10,8,29,5.0,33.0,22.0,18.31,12.0,Bad,3571.70,27.03,6.5,Yes,60.96,50.85,High_spent_Small_value_payments,341.18
49998,25,Mechanic,39628.99,4,6,7,2.0,21.0,6.0,11.50,7.0,Good,502.38,39.14,32.1,No,35.10,97.60,High_spent_Small_value_payments,463.24


In [31]:
# get a picture of Payment_of_Min_Amount

column = 'Payment_of_Min_Amount'

# value counts
values_counted = df_test[column].value_counts()

# filter out values with counts >1
filtered_counts = values_counted[values_counted > 3]

filtered_counts

Payment_of_Min_Amount
Yes    14335
No     10591
NM      3434
Name: count, dtype: int64

In [32]:
# drop NM from Payment_of_Min_Amount
df_test = df_test[df_test['Payment_of_Min_Amount'] != 'NM']

In [33]:
# get a picture of Outstanding_Debt

column = 'Outstanding_Debt'

# value counts
values_counted = df_test[column].value_counts()

# filter out values with counts >1
filtered_counts = values_counted[values_counted > 3]

filtered_counts

Outstanding_Debt
573.77     8
1360.45    8
413.08     7
2530.46    7
2646.00    7
          ..
1264.51    4
413.34     4
1082.13    4
1219.91    4
1163.09    4
Name: count, Length: 1068, dtype: int64

In [34]:
# cast some columns as int when appropriate

df_test['Num_of_Delayed_Payment'] = df_test['Num_of_Delayed_Payment'].astype(int)
df_test['Num_Credit_Inquiries'] = df_test['Num_Credit_Inquiries'].astype(int)
df_test['Num_of_Loan'] = df_test['Num_of_Loan'].astype(int)
df_test['Delay_from_due_date'] = df_test['Delay_from_due_date'].astype(int)


/var/folders/zt/x756b70j3v75355fl48s3dzc0000gn/T/ipykernel_7382/171639993.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['Num_of_Delayed_Payment'] = df_test['Num_of_Delayed_Payment'].astype(int)
/var/folders/zt/x756b70j3v75355fl48s3dzc0000gn/T/ipykernel_7382/171639993.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['Num_Credit_Inquiries'] = df_test['Num_Credit_Inquiries'].astype(int)
/var/folders/zt/x756b70j3v75355fl48s3dzc0000gn/T/ipykernel_7382/171639993.py:5: SettingWithCopyWarni

In [35]:
# get a picture of Delay_from_due_date

column = 'Delay_from_due_date'

# value counts
values_counted = df_test[column].value_counts()

# filter out values with counts >1
filtered_counts = values_counted[values_counted > 3]

filtered_counts

Delay_from_due_date
 13    895
 15    893
 11    854
 10    845
 8     843
      ... 
 64     14
 63     12
 65     12
-5       7
 67      6
Name: count, Length: 72, dtype: int64

In [36]:
# get a picture of Annual_Income

column = 'Annual_Income'

# value counts
values_counted = df_test[column].describe()
values_counted


count    2.492600e+04
mean     1.606978e+05
std      1.314917e+06
min      7.006035e+03
25%      1.946292e+04
50%      3.748569e+04
75%      7.304456e+04
max      2.413726e+07
Name: Annual_Income, dtype: float64

In [37]:
#  find and remove outlier, typo rows - Annual_Income

# Define the range of normal values
lower_bound = 20000
upper_bound = 1000000

column = 'Annual_Income'

# testing variable
within_range_df = df_test[(df_test[column] >= lower_bound) & (df_test[column] <= upper_bound)]

# used this to play with range and dtm where cutoff for normal values were
outliers_df = df_test[(df_test[column] < lower_bound) | (df_test[column] > upper_bound)]

# Filter the DataFrame to keep only rows with values within the specified range
cleaned_df = df_test[(df_test[column] >= lower_bound) & (df_test[column] <= upper_bound)]

df_test = cleaned_df

df_test

,Age,Occupation,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
4,28,_______,34847.84,2,4,6,1,3,1,5.42,5,Good,605.03,25.93,27.2,No,18.82,39.68,High_spent_Large_value_payments,485.30
5,28,Teacher,34847.84,2,4,6,1,3,3,5.42,5,Good,605.03,30.12,27.3,No,18.82,251.63,Low_spent_Large_value_payments,303.36
9,35,Engineer,143162.64,1,5,8,3,6,3,2.10,3,Good,1303.01,35.69,18.5,No,246.99,453.62,Low_spent_Large_value_payments,788.11
12,55,Entrepreneur,30689.89,2,5,4,1,5,6,1.99,4,Good,632.46,39.38,17.9,No,16.42,69.11,High_spent_Medium_value_payments,425.73
14,55,Entrepreneur,30689.89,2,5,4,1,5,6,1.99,4,Good,632.46,36.82,18.1,No,16.42,191.89,Low_spent_Medium_value_payments,332.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49992,29,Architect,20002.88,10,8,29,5,33,25,18.31,9,Bad,3571.70,32.39,6.3,Yes,60.96,107.21,Low_spent_Small_value_payments,314.82
49993,29,Architect,20002.88,10,8,29,5,33,25,18.31,12,Bad,3571.70,37.53,6.4,Yes,60.96,71.79,Low_spent_Small_value_payments,350.23
49994,29,_______,20002.88,10,8,29,5,33,22,18.31,12,Bad,3571.70,27.03,6.5,Yes,60.96,50.85,High_spent_Small_value_payments,341.18
49998,25,Mechanic,39628.99,4,6,7,2,21,6,11.50,7,Good,502.38,39.14,32.1,No,35.10,97.60,High_spent_Small_value_payments,463.24


In [38]:
# get a picture of Delay_from_due_date

column = 'Num_Credit_Inquiries'

# value counts
values_counted = df_test[column].value_counts()

# filter out values with counts >1
filtered_counts = values_counted[values_counted > 3]

filtered_counts

Num_Credit_Inquiries
5     2065
4     1962
6     1751
3     1576
7     1572
8     1414
2     1153
9     1120
10     868
11     830
1      795
12     713
13     518
0      463
14     346
15     271
16     207
17      92
Name: count, dtype: int64

In [39]:
#  find and remove outlier, typo rows - Annual_Income

# Define the range of normal values
lower_bound = 0
upper_bound = 20

column = 'Num_Credit_Inquiries'

# testing variable
within_range_df = df_test[(df_test[column] >= lower_bound) & (df_test[column] <= upper_bound)]

# used this to play with range and dtm where cutoff for normal values were
outliers_df = df_test[(df_test[column] < lower_bound) | (df_test[column] > upper_bound)]

# Filter the DataFrame to keep only rows with values within the specified range
cleaned_df = df_test[(df_test[column] >= lower_bound) & (df_test[column] <= upper_bound)]

df_test = cleaned_df

df_test

,Age,Occupation,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
4,28,_______,34847.84,2,4,6,1,3,1,5.42,5,Good,605.03,25.93,27.2,No,18.82,39.68,High_spent_Large_value_payments,485.30
5,28,Teacher,34847.84,2,4,6,1,3,3,5.42,5,Good,605.03,30.12,27.3,No,18.82,251.63,Low_spent_Large_value_payments,303.36
9,35,Engineer,143162.64,1,5,8,3,6,3,2.10,3,Good,1303.01,35.69,18.5,No,246.99,453.62,Low_spent_Large_value_payments,788.11
12,55,Entrepreneur,30689.89,2,5,4,1,5,6,1.99,4,Good,632.46,39.38,17.9,No,16.42,69.11,High_spent_Medium_value_payments,425.73
14,55,Entrepreneur,30689.89,2,5,4,1,5,6,1.99,4,Good,632.46,36.82,18.1,No,16.42,191.89,Low_spent_Medium_value_payments,332.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49992,29,Architect,20002.88,10,8,29,5,33,25,18.31,9,Bad,3571.70,32.39,6.3,Yes,60.96,107.21,Low_spent_Small_value_payments,314.82
49993,29,Architect,20002.88,10,8,29,5,33,25,18.31,12,Bad,3571.70,37.53,6.4,Yes,60.96,71.79,Low_spent_Small_value_payments,350.23
49994,29,_______,20002.88,10,8,29,5,33,22,18.31,12,Bad,3571.70,27.03,6.5,Yes,60.96,50.85,High_spent_Small_value_payments,341.18
49998,25,Mechanic,39628.99,4,6,7,2,21,6,11.50,7,Good,502.38,39.14,32.1,No,35.10,97.60,High_spent_Small_value_payments,463.24


In [40]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17716 entries, 4 to 49999
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       17716 non-null  int64  
 1   Occupation                17716 non-null  object 
 2   Annual_Income             17716 non-null  float64
 3   Num_Bank_Accounts         17716 non-null  int64  
 4   Num_Credit_Card           17716 non-null  int64  
 5   Interest_Rate             17716 non-null  int64  
 6   Num_of_Loan               17716 non-null  int64  
 7   Delay_from_due_date       17716 non-null  int64  
 8   Num_of_Delayed_Payment    17716 non-null  int64  
 9   Changed_Credit_Limit      17716 non-null  float64
 10  Num_Credit_Inquiries      17716 non-null  int64  
 11  Credit_Mix                17716 non-null  object 
 12  Outstanding_Debt          17716 non-null  float64
 13  Credit_Utilization_Ratio  17716 non-null  float64
 14  Credit_Hist

In [41]:
# Export the DataFrame to a CSV file
df_test.to_csv('cleaned_data_test.csv', index=False)